# Backtrader

En esta notebook realizamos una simulacion utilizando los modelos entrenados

In [1]:
! pip install backtrader;

In [2]:
import backtrader as bt
import pandas as pd

Importamos la data de test de los modelos, en todos los casos es la misma data excluyendo el target, consta del 2018 al 2020. Estos son datos que los modelos no han visto.

In [3]:
data = pd.read_csv('models/model_test_Forward_Return_1m.csv', index_col = 0, parse_dates=True)
data.head(3)

,Open,High,Low,Close,Volume,MACD,RSI,BB_High,BB_Mid,BB_Low,ATR,NATR,Currency_Volume,Adj Close,Month,Weekday,Ticker,Currency,Forward_Return_1m
Date,,,,,,,,,,,,,,,,,,,
2018-07-24,360.50,360.50,360.50,360.50,0.0,-1.705303e-13,0.000000,5.890262,5.890262,5.890262,3.526296e-25,9.781681e-26,0.000000e+00,358.496796,7,1,AMGN.BA,ARS,0.000000
2018-07-24,4.99,5.06,4.93,4.99,29389000.0,-3.568515e-02,54.042247,1.795840,1.753049,1.710258,1.408903e-01,2.823454e+00,1.466511e+08,4.766606,7,1,ABEV,USD,-0.118236
2018-07-24,5.64,5.74,5.62,5.69,546800.0,-1.256564e-01,49.939444,1.926558,1.881268,1.835979,1.809786e-01,3.180644e+00,3.111292e+06,5.248040,7,1,ELP,USD,-0.128295


In [4]:
data.index.is_monotonic, data.index[0], data.index[-1]

(True, Timestamp('2018-07-24 00:00:00'), Timestamp('2020-06-10 00:00:00'))

Derivo un dataframe para usar en la simulacion que no contiene el target, ademas, convierto a variable numerica las variables de tipo string porque no son aceptadas por backtrader, podemos convertirlas de vuelta para mandarla al modelo.

In [26]:
from sklearn.preprocessing import LabelEncoder

simulation_data = data.drop('Forward_Return_1m', axis = 1)[:500]

ticker_encoder = LabelEncoder()
simulation_data['Ticker_Name'] = simulation_data['Ticker']
simulation_data['Ticker'] = ticker_encoder.fit_transform(simulation_data.Ticker)

currency_encoder = LabelEncoder()
simulation_data['Currency'] = currency_encoder.fit_transform(simulation_data.Currency)

In [27]:
class PandasData(bt.feeds.PandasData):
    lines = (
        'macd',
        'rsi',
        'bb_high',
        'bb_mid',
        'bb_low',
        'atr',
        'natr',
        'currency_volume',
        'adj_close',
        'month',
        'weekday',
        'ticker',
        'currency'
    )
    params = (
        ('datetime', None),
        ('open', 0),
        ('high', 1),
        ('low', 2),
        ('close', 3),
        ('volume', 4),
        ('macd', 5),
        ('rsi', 6),
        ('bb_high', 7),
        ('bb_mid', 8),
        ('bb_low', 9),
        ('atr', 10),
        ('natr', 11),
        ('currency_volume', 12),
        ('adj_close', 13),
        ('month', 14),
        ('weekday', 15),
        ('ticker', 16),
        ('currency', 17)
    )

In [28]:
test = []

In [36]:
import numpy as np

class TestStrategy(bt.Strategy):
    def __init__(self): pass
        #self.open = [x.open for x in self.datas]
        #self.high = [x.high for x in self.datas]
        #self.low = [x.low for x in self.datas]
        #self.close = [x.close for x in self.datas]
        #self.volume = [x.volume for x in self.datas]
        #self.macd = [x.macd for x in self.datas]
        #self.rsi = [x.rsi for x in self.datas]
        #self.bb_high = [x.bb_high for x in self.datas]
        #self.bb_mid = [x.bb_mid for x in self.datas]
        #self.bb_low = [x.bb_low for x in self.datas]
        #self.atr = [x.atr for x in self.datas]
        #self.natr = [x.natr for x in self.datas]
        #self.currency_volume = [x.currency_volume for x in self.datas]
        #self.adj_close = [x.adj_close for x in self.datas]
        #self.month = [x.month for x in self.datas]
        #self.weekday = [x.weekday for x in self.datas]
        #self.ticker = [x.ticker for x in self.datas]
        #self.currency = [x.currency for x in self.datas]
    def next(self):
        for i, d in enumerate(self.datas):
            dt, dn = self.datetime.date(), d._name
            pos = self.getposition(d).size
            if dn == 'PBI': print('{} {} {} Position {}'.format(dt, dn, d.close[0], pos))

In [37]:
cerebro = bt.Cerebro()
cerebro.broker.setcash(100000.0)


for ticker in simulation_data.Ticker_Name.unique():
    ticker_mask = simulation_data.Ticker_Name == ticker
    ticker_data = simulation_data.loc[ticker_mask]
    cerebro.adddata(PandasData(dataname=ticker_data), name = ticker)

cerebro.addstrategy(TestStrategy)

print('Starting Portfolio Value: %.2f' % cerebro.broker.getvalue())
cerebro.run()
print('Final Portfolio Value: %.2f' % cerebro.broker.getvalue())

Starting Portfolio Value: 100000.00
2018-07-25 PBI 8.680000305175781 Position 0
2018-07-26 PBI 8.84000015258789 Position 0
Final Portfolio Value: 100000.00


In [38]:
simulation_data[['Ticker_Name', 'Close']][simulation_data.Ticker_Name == 'PBI']

,Ticker_Name,Close
Date,,
2018-07-25,PBI,8.68
2018-07-26,PBI,8.84
